# <b> Project 1

<b>Introduction<br></b>
In this project, there will be three parts to analyze pandas. First part will be Grammar for pandas library. Second part will implement an algorithm to identify if the input is vaild DataFrame structure. Third part will be the cost analysis of DataFrame and ___ . 

The following BNF grammar definition of Pandas:


\begin{eqnarray*}
\textit{ mod}& ::= &\textbf{Module(stmt* body))} \\
                 &  |  &\textbf{Expression(expr body)} \\
\\
\textit{stmt}& ::= &\textbf{Assign(expr* targets, stmt value))} \\
\\
\\
\textit{expr}& ::= &\textbf{Attribute(expr value, identifier attr, expr_context ctx)} \\
                 &  |  &\textbf{ Num(object n)} \\
                 &  |  &\textbf{ Subscript(expr value, slice slice, expr_context ctx)} \\
                 &  |  &\textbf{ Str(string s)} \\
                 &  |  &\textbf{ Name(identifier id, expr_context ctx)} \\
                 &  |  &\textbf{ Dict(expr* keys,  expr* values)} \\
                 &  |  &\textbf{List(expr* elts, expr_context ctx)} \\
                 &  |  &\textbf{Tuple(expr* elts, expr_context ctx)} \\
                 &  |  &\textbf{Slice(expr* value)} \\
                 &  |  &\textbf{Call(expr func, expr* args, keyword* keywords)} \\
                 &  |  &\textbf{BinOp(expr left, operator op, expr right)} \\
                 \\
                 \\
\textit{operator}& ::= &\textbf{Add } \\
\\
\\
\textit{keyword arguments supplied to call:}\\
\textit{keyword}& ::= &\textbf{(identifier? arg, expr value) } \\
\end{eqnarray*}


<b>Part II</b>
<p>Below is an implementation of an algorithm that determine if the input statement can be transform or is already DataFrame or not
<br>

In [362]:
#This 
import ast 
import pandas as pd
class is_DataFrame(ast.NodeVisitor):
    def visit_Module(self, node):
        results = [self.visit(s) for s in node.body]
        return all(results)

    def visit_Expr(self, node):
        return self.visit(node.value)
    
    def visit_Call(self, node):
        results = [self.visit(s) for s in node.keywords]
        return all(results)
    
    def visit_keyword(self, node):
        return self.visit(node.value)
    
    def visit_BinOp(self, node):
        return self.visit(node.left) and self.visit(node.right)

    def visit_List(self, node):
        return True
    
    def visit_Dict(self, node):
        '''need to specify the if index exist'''
        return True
    
    def visit_Subscript(self, node):
        return True
    


test = ast.parse('pd.DataFrame(data = [1,2,3] + [9,2,3] )')
test1 = ast.parse('pd.DataFrame(data = 1)')
print(is_DataFrame().visit(test))
print(is_DataFrame().visit(test1))

True
False


<b>Part III</b>
<p>a) Implementation of a static analysis algorithm that infers the type of a single DataFrame expression.

In [376]:
import ast 
import pandas as pd

class infer_type(ast.NodeVisitor):
    
    def visit_Module(self, node):
        results = [self.visit(s) for s in node.body]
        if results == [dict] or results == [list]:
            return results[0]

    def visit_Expr(self, node):
        return self.visit(node.value)

    def visit_Call(self, node):
        results = [self.visit(s) for s in node.keywords]
        if results == [dict] or results == [list]:
            return results[0]
    
    def visit_keyword(self, node):
        return self.visit(node.value)

    def visit_BinOp(self, node):
        if type(node.op) in [ast.Add]:
            if self.visit(node.left) is list and\
               self.visit(node.right) is list:
                return list    
    
    def visit_List(self, node):
        return list
    
    def visit_Dict(self, node):
        return dict


In [462]:
def typeInput(newInput):
    '''test your input if is a dataframe then try to 
        find out the type of you dataframe parameter'''
    test = ast.parse(newInput)
    res = is_DataFrame().visit(test)
    if res == True:
        return infer_type().visit(test)
    else:
        return "Your input is not a valid DataFrame input"
    

    

a1 = 'pd.DataFrame(data = {1:1,2:2},index={0,1})'    
a = ast.parse(a1)
print(len(a.body[0].value.keywords))
ast.dump(a)
print(ast.dump(a))
# print(typeInput(a1))

2
Module(body=[Expr(value=Call(func=Attribute(value=Name(id='pd', ctx=Load()), attr='DataFrame', ctx=Load()), args=[], keywords=[keyword(arg='data', value=Dict(keys=[Num(n=1), Num(n=2)], values=[Num(n=1), Num(n=2)])), keyword(arg='index', value=Set(elts=[Num(n=0), Num(n=1)]))]))])


<p>b) DataFrame Cost analysis without runing code

In [453]:
class arr_memory_check(ast.NodeVisitor):
    
    def __init__(self):
        self.nn = 0
        self.single = 1
        self.mm = 1
    
    def visit_Module(self, node):
        results = [self.visit(s) for s in node.body]
        return results[0]

    def visit_Expr(self, node):
        return self.visit(node.value)

    def visit_Call(self, node):
        results = [self.visit(s) for s in node.keywords]
        return results[0]
    
    def visit_keyword(self, node):
        return self.visit(node.value)

    def visit_BinOp(self, node):
         return self.visit(node.left) + self.visit(node.right)    
    
    def visit_List(self, node):
        self.nn = len(node.elts)
        result = [self.visit(s) for s in node.elts]
        return result[0]
    
    def visit_Dict(self, node):
        self.nn = len(node.keys)
        result = [self.visit(s) for s in node.values]
        return result[0]
    
    def visit_Num(self, node):
        self.single = 1
        return self.nn * self.single * self.mm

    
a1 = 'pd.DataFrame(data = {1:[2,8],2:[9,3]})'    
a = ast.parse(a1)
ast.dump(a)
print(ast.dump(a))
print(arr_memory_check().visit(a))

Module(body=[Expr(value=Call(func=Attribute(value=Name(id='pd', ctx=Load()), attr='DataFrame', ctx=Load()), args=[], keywords=[keyword(arg='data', value=Dict(keys=[Num(n=1), Num(n=2)], values=[List(elts=[Num(n=2), Num(n=8)], ctx=Load()), List(elts=[Num(n=9), Num(n=3)], ctx=Load())]))]))])
2


In [452]:
pd.DataFrame(data = {1:1,2:2},index={0,1})

,1,2
0,1,2
1,1,2
